In [ ]:
# Databento 概览

Databento 文档: 

* [https://databento.com/docs](https://databento.com/docs)

## 3 种服务

Databento 提供 3 种类型的服务：

1. `Historical` - 用于超过24小时的历史市场数据
2. `Live` - 用于最近24小时内的市场数据
3. `Reference` - 用于证券主文件和公司行为数据

## 3 种文件格式

Databento 支持 3 种数据格式：

* `DBN` - Databento二进制编码（二进制格式）
* `csv` - 逗号分隔值（文本格式）
* `json` - JavaScript对象表示法（文本格式）

## Python 库

Databento 提供简单的 Python 库（本教程中使用）：

`pip install -U databento`


In [ ]:
## 架构

架构（Schema）只是对`数据类型`的一个专业称呼。

按详细程度排序的最常用架构：

| 架构 | 类型 | 描述 |
|--------|------|-------------|
| `mbo` | L3 数据 | 提供所有价格级别的每个订单簿事件，按订单ID键索引。允许确定每个订单的队列位置，提供最高级别的粒度。 |
| `mbp-10` | L2 数据 | 提供前十个价格级别的每个订单簿事件，按价格键索引。包括交易和聚合市场深度变化，具有前十个价格级别的总规模和订单数量。 |
| `mbp-1` | L1 数据 | 提供更新顶级价格级别（BBO）的每个订单簿事件。包括交易和簿深度变化，具有BBO的总规模和订单数量。 |
| `bbo-1s` | L1 采样 | 类似于L1数据，但以1秒间隔采样。以1秒间隔提供最后最佳买价、最佳卖价和销售价格。 |
| `tbbo` | L1 交易 | 提供每个交易事件以及每个交易影响前的BBO。是MBP-1的子集。 |
| `trades` | 交易数据 | 提供每个交易事件。这是MBO数据的子集。 |
| `ohlcv-1s` | 1秒K线 | 从1秒间隔的交易聚合的OHLCV K线。 |
| `ohlcv-1m` | 1分钟K线 | 从1分钟间隔的交易聚合的OHLCV K线。 |
| `ohlcv-1h` | 1小时K线 | 从1小时间隔的交易聚合的OHLCV K线。 |
| `ohlcv-1d` | 1日K线 | 从1日间隔的交易聚合的OHLCV K线。 |
| `definition` | 参考 | 提供关于工具的参考信息，包括符号、名称、到期日、上市日、最小价格变动、执行价格。 |
| `status` | 交易所状态 | 提供关于交易时段的更新，如暂停、暂停、限制卖空、拍卖开始和其他撮合引擎状态。 |
| `statistics` | 交易所统计 | 提供场所发布的官方汇总统计信息，包括日成交量、未平仓量、结算价格和官方开/高/低价格。 |

**Databento 如何生成低分辨率数据？**

1. Databento 首先从每个来源收集最详细的可用市场数据（如果可用，主要是`mbo`）
2. 然后从这个最精细的数据中导出所有其他格式，以确保所有数据类型（架构）之间100%的一致性。

其他资源：

* 如何将tick/交易数据转换为K线的示例教程：
    * [https://databento.com/docs/examples/basics-historical/tick-resampling/example](https://databento.com/docs/examples/basics-historical/tick-resampling/example)
* 详细解释的所有架构：
    * [https://databento.com/docs/schemas-and-data-formats?historical=python&live=python&reference=python](https://databento.com/docs/schemas-and-data-formats?historical=python&live=python&reference=python)


In [ ]:
## 符号体系

符号体系（Symbology）只是对各种工具命名约定的专业称呼。在API和文档中经常使用缩写`stypes`，意思是"符号体系类型"。

Databento 支持 5 种符号体系（命名约定）：

| 符号体系类型    | 描述                                      | 示例/模式                | 关键注意事项                                                                   |
|:-----------------|:-------------------------------------------------|:------------------------------|:----------------------------------------------------------------------------|
| `raw_symbol`     | 数据发布商使用的原始字符串符号    | `AAPL`, `ESH3`                | 最适合直接市场连接环境                             |
| `instrument_id`  | 发布商分配的唯一数字ID          | `12345`, `9876543`            | 空间高效，但某些发布商可能每天重新映射                 |
| `parent`         | 使用根符号对相关符号进行分组          | `ES.FUT`, `ES.OPT`            | 允许一次查询根符号的所有期货/期权                |
| `continuous`     | 引用随时间变化的工具      | `ES.c.0`, `CL.n.1`, `ZN.v.0`  | 滚动规则：日历(c)、未平仓量(n)、成交量(v)                      |
| `ALL_SYMBOLS`    | 请求数据集中的所有符号                  | `ALL_SYMBOLS`                  |    |


请求数据时，可以指定**输入**和**输出**符号体系。支持这4种组合（针对各种交易所/发布商）：

| 输入 SType    | 输出 SType      | DBEQ.BASIC | GLBX.MDP3 | IFEU.IMPACT | NDEX.IMPACT | OPRA.PILLAR | XNAS.ITCH |
|:---------------|:-----------------|:-----------|:----------|:------------|:------------|:------------|:----------|
| `parent`       | `instrument_id`  |            | ✓         | ✓           | ✓           | ✓           |           |
| `continuous`   | `instrument_id`  |            | ✓         |             |             |             |           |
| `raw_symbol`   | `instrument_id`  | ✓          | ✓         | ✓           | ✓           | ✓           | ✓         |
| `instrument_id`| `raw_symbol`     | ✓          | ✓         | ✓           | ✓           | ✓           | ✓         |

更多详情：

* [https://databento.com/docs/standards-and-conventions/symbology?historical=python&live=python&reference=python](https://databento.com/docs/standards-and-conventions/symbology?historical=python&live=python&reference=python)


In [ ]:
## Databento 文件格式

Databento 使用自己的市场数据文件格式，称为 **Databento二进制编码（DBN）**。
可以将其视为CSV/JSON文件的更高性能+压缩替代方案。

您可以轻松加载DBN文件并将其转换为简单的CSV/JSON数据。

更多详情：

* [https://databento.com/docs/standards-and-conventions/databento-binary-encoding#getting-started-with-dbn?historical=python&live=python&reference=python](https://databento.com/docs/standards-and-conventions/databento-binary-encoding#getting-started-with-dbn?historical=python&live=python&reference=python)


In [ ]:
# 历史API示例


In [ ]:
## 认证与连接到 Databento


In [ ]:
import databento as db


# 建立连接并认证
API_KEY = "db-8VWGBis54s4ewGVciMRakNxLCJKen"   # 在这里放入您的API密钥（现有密钥只是示例，非真实）
client = db.Historical(API_KEY)


In [ ]:
## 元数据

### 列出发布商

显示所有数据发布商。


In [ ]:
publishers = client.metadata.list_publishers()

# 从长列表中只显示前五个
publishers[:5]


In [ ]:
示例输出：

```python
[{'publisher_id': 1,
  'dataset': 'GLBX.MDP3',
  'venue': 'GLBX',
  'description': 'CME Globex MDP 3.0'},
 {'publisher_id': 2,
  'dataset': 'XNAS.ITCH',
  'venue': 'XNAS',
  'description': 'Nasdaq TotalView-ITCH'},
 {'publisher_id': 3,
  'dataset': 'XBOS.ITCH',
  'venue': 'XBOS',
  'description': 'Nasdaq BX TotalView-ITCH'},
 {'publisher_id': 4,
  'dataset': 'XPSX.ITCH',
  'venue': 'XPSX',
  'description': 'Nasdaq PSX TotalView-ITCH'},
 {'publisher_id': 5,
  'dataset': 'BATS.PITCH',
  'venue': 'BATS',
  'description': 'Cboe BZX Depth Pitch'}]
```


In [ ]:
### 列出数据集

每个数据集的格式为：`发布商.数据集`

* 发布商/市场代码基于：[https://www.iso20022.org/market-identifier-codes](https://www.iso20022.org/market-identifier-codes)


In [ ]:
datasets = client.metadata.list_datasets()
datasets


In [ ]:
示例输出：

```python
['ARCX.PILLAR',
 'DBEQ.BASIC',
 'EPRL.DOM',
 'EQUS.SUMMARY',
 'GLBX.MDP3',
 'IEXG.TOPS',
 'IFEU.IMPACT',
 'NDEX.IMPACT',
 'OPRA.PILLAR',
 'XASE.PILLAR',
 'XBOS.ITCH',
 'XCHI.PILLAR',
 'XCIS.TRADESBBO',
 'XNAS.BASIC',
 'XNAS.ITCH',
 'XNYS.PILLAR',
 'XPSX.ITCH']
```


In [ ]:
### 列出架构

列出Databento中所有支持的数据格式。


In [ ]:
schemas = client.metadata.list_schemas(dataset="GLBX.MDP3")
schemas


In [ ]:
示例输出：

```python
['mbo',
 'mbp-1',
 'mbp-10',
 'tbbo',
 'trades',
 'bbo-1s',
 'bbo-1m',
 'ohlcv-1s',
 'ohlcv-1m',
 'ohlcv-1h',
 'ohlcv-1d',
 'definition',
 'statistics',
 'status']
```


In [ ]:
### 数据集状况

显示数据可用性和质量。


In [ ]:
conditions = client.metadata.get_dataset_condition(
    dataset="GLBX.MDP3",
    start_date="2022-06-06",
    end_date="2022-06-10",
)

conditions


In [ ]:
示例输出：

```python
[{'date': '2022-06-06',
  'condition': 'available',
  'last_modified_date': '2024-05-18'},
 {'date': '2022-06-07',
  'condition': 'available',
  'last_modified_date': '2024-05-21'},
 {'date': '2022-06-08',
  'condition': 'available',
  'last_modified_date': '2024-05-21'},
 {'date': '2022-06-09',
  'condition': 'available',
  'last_modified_date': '2024-05-21'},
 {'date': '2022-06-10',
  'condition': 'available',
  'last_modified_date': '2024-05-22'}]
```


In [ ]:
### 数据集范围

显示数据集的可用范围。

* 使用此方法发现数据可用性。
* 响应中的开始和结束值可以与`timeseries.get_range`和`batch.submit_job`端点一起使用。


In [ ]:
available_range = client.metadata.get_dataset_range(dataset="GLBX.MDP3")
available_range


In [ ]:
示例输出：

```python
{'start': '2010-06-06T00:00:00.000000000Z',
 'end': '2025-01-18T00:00:00.000000000Z'}
```


In [ ]:
### 记录计数

返回数据查询返回的记录数量。


In [ ]:
record_count = client.metadata.get_record_count(
    dataset="GLBX.MDP3",
    symbols=["ESM2"],   # ES（标普合约）2022年6月到期
    schema="ohlcv-1h",  # 1小时K线；只能使用10分钟倍数的时间范围（不能用于1分钟K线）
    start="2022-01-06", # 包括开始时间
    end="2022-01-07"    # 不包括结束时间
)

# 交易所有一个小时的休息时间，所以23个小时K线是正常的
record_count


In [ ]:
示例输出：

`23`


In [ ]:
### 费用

获取费用 = 您为数据支付的美元费用。


In [ ]:
cost = client.metadata.get_cost(
    dataset="GLBX.MDP3",
    symbols=["ESM2"],
    schema="ohlcv-1h",  # 1小时K线；只能使用10分钟倍数的时间范围（不能用于1分钟K线）
    start="2022-01-06", # 包括开始时间
    end="2022-01-07"    # 不包括结束时间
)

cost


In [ ]:
示例输出：

`0.00022791326`


In [ ]:
## 时间序列数据

### `get_range`

* 向Databento发送流式时间序列数据请求。
* 这是将历史市场数据、工具定义和状态数据直接获取到应用程序中的主要方法。
* 此方法仅在下载完所有数据后返回，这可能需要很长时间。

**警告:**
* `ts_event`表示聚合的开始时间。所以如果我们下载K线，时间戳表示每个K线的**开盘时间**。


In [ ]:
data = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    symbols=["ESM2"],            # ES（标普合约）2022年6月到期
    schema="ohlcv-1h",           # 小时K线
    start="2022-06-01T00:00:00",
    end="2022-06-03T00:10:00",
    limit=5,                    # 可选的结果数量限制
)

# 数据以DBNStore格式接收
data


In [ ]:
示例输出：

`<DBNStore(schema=ohlcv-1h)>`


In [ ]:
# 将DBN格式转换为pandas数据框
df = data.to_df()

# 预览
print(len(df))
df


In [ ]:
示例输出：*（非真实数据，只是输出格式示例）*

| ts_event | rtype | publisher_id | instrument_id | open | high | low | close | volume | symbol |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| 2022-06-01 00:00:00+00:00 | 34 | 1 | 3403 | 4149.25 | 4153.50 | 4149.00 | 4150.75 | 9281 | ESM2 |
| 2022-06-01 01:00:00+00:00 | 34 | 1 | 3403 | 4151.00 | 4157.75 | 4149.50 | 4154.25 | 11334 | ESM2 |
| 2022-06-01 02:00:00+00:00 | 34 | 1 | 3403 | 4154.25 | 4155.25 | 4146.50 | 4147.00 | 7258 | ESM2 |


In [ ]:
注意：

* `rtype` = 1小时K线 
* 更多类似代码：[https://databento.com/docs/standards-and-conventions/common-fields-enums-types#rtype?historical=python&live=python&reference=python](https://databento.com/docs/standards-and-conventions/common-fields-enums-types#rtype?historical=python&live=python&reference=python)


In [ ]:
## 符号

### `resolve`

将符号列表从**输入**符号体系类型解析为**输出**符号体系类型。

* 示例：`raw_symbol`到`instrument_id`：`ESM2` → `3403`


In [ ]:
result = client.symbology.resolve(
    dataset="GLBX.MDP3",
    symbols=["ESM2"],
    stype_in="raw_symbol",
    stype_out="instrument_id",
    start_date="2022-06-01",
    end_date="2022-06-30",
)

result


In [ ]:
示例输出：

```python
{'result': {'ESM2': [{'d0': '2022-06-01', 'd1': '2022-06-26', 's': '3403'}]},
 'symbols': ['ESM2'],
 'stype_in': 'raw_symbol',
 'stype_out': 'instrument_id',
 'start_date': '2022-06-01',
 'end_date': '2022-06-30',
 'partial': [],
 'not_found': [],
 'message': 'OK',
 'status': 0}
```


In [ ]:
最重要的是`result`和键值对`'s': '3403'`，它包含instrument_id的值。


In [ ]:
## DBNStore 操作

`DBNStore` 对象是用于处理 `DBN` 编码数据的辅助类。


In [ ]:
### `from_bytes`

从 DBN 字节流读取数据。


In [ ]:
dbn_data = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    symbols=["ESM2"],
    schema="ohlcv-1h",
    start="2022-06-06",
    limit=3
)

dbn_data.to_df()


In [ ]:
示例输出：*（非真实数据，只是输出格式示例）*

| ts_event | rtype | publisher_id | instrument_id | open | high | low | close | volume | symbol |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| 2022-06-06 00:00:00+00:00 | 34 | 1 | 3403 | 4109.50 | 4117.00 | 4105.50 | 4115.75 | 8541 | ESM2 |
| 2022-06-06 01:00:00+00:00 | 34 | 1 | 3403 | 4115.75 | 4122.75 | 4113.00 | 4122.25 | 14008 | ESM2 |
| 2022-06-06 02:00:00+00:00 | 34 | 1 | 3403 | 4122.25 | 4127.00 | 4120.75 | 4126.25 | 10150 | ESM2 |


In [ ]:
# 将流数据保存到文件 - 推荐的后缀是：`*.dbn.zst`
path = "./GLBX-ESM2-20220606.ohlcv-1h.dbn.zst"
dbn_data.to_file(path)


In [ ]:
# 从之前保存的文件加载数据并重新创建 DBN 对象
with open(path, "rb") as saved:
    loaded_dbn_data = db.DBNStore.from_bytes(saved)

loaded_dbn_data.to_df()


In [ ]:
示例输出 *（非真实数据，只是输出格式示例）*：

| ts_event | rtype | publisher_id | instrument_id | open | high | low | close | volume | symbol |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| 2022-06-06 00:00:00+00:00 | 34 | 1 | 3403 | 4109.50 | 4117.00 | 4105.50 | 4115.75 | 8541 | ESM2 |
| 2022-06-06 01:00:00+00:00 | 34 | 1 | 3403 | 4115.75 | 4122.75 | 4113.00 | 4122.25 | 14008 | ESM2 |
| 2022-06-06 02:00:00+00:00 | 34 | 1 | 3403 | 4122.25 | 4127.00 | 4120.75 | 4126.25 | 10150 | ESM2 |


In [ ]:
### `from_file`

从 DBN 文件读取数据。


In [ ]:
loaded_dbn_data = db.DBNStore.from_file(path)
loaded_dbn_data.to_df()


In [ ]:
示例输出：*（非真实数据，只是输出格式示例）*

| ts_event | rtype | publisher_id | instrument_id | open | high | low | close | volume | symbol |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| 2022-06-06 00:00:00+00:00 | 34 | 1 | 3403 | 4109.50 | 4117.00 | 4105.50 | 4115.75 | 8541 | ESM2 |
| 2022-06-06 01:00:00+00:00 | 34 | 1 | 3403 | 4115.75 | 4122.75 | 4113.00 | 4122.25 | 14008 | ESM2 |
| 2022-06-06 02:00:00+00:00 | 34 | 1 | 3403 | 4122.25 | 4127.00 | 4120.75 | 4126.25 | 10150 | ESM2 |


In [ ]:
### `to_csv`

将数据写入 CSV 格式文件。


In [ ]:
dbn_data = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    symbols=["ESM2"],
    schema="ohlcv-1h",
    start="2022-06-06",
    limit=3
)

# 导出为 CSV 文件
dbn_data.to_csv("GLBX-ESM2-20220606-ohlcv-1h.csv")


In [ ]:
### `to_df`

将 DBN 数据转换为 pandas DataFrame。


In [ ]:
# 导出为 pandas DataFrame
dbn_data.to_df()


In [ ]:
示例输出：*（非真实数据，只是输出格式示例）*

| ts_event | rtype | publisher_id | instrument_id | open | high | low | close | volume | symbol |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| 2022-06-06 00:00:00+00:00 | 34 | 1 | 3403 | 4109.50 | 4117.00 | 4105.50 | 4115.75 | 8541 | ESM2 |
| 2022-06-06 01:00:00+00:00 | 34 | 1 | 3403 | 4115.75 | 4122.75 | 4113.00 | 4122.25 | 14008 | ESM2 |
| 2022-06-06 02:00:00+00:00 | 34 | 1 | 3403 | 4122.25 | 4127.00 | 4120.75 | 4126.25 | 10150 | ESM2 |


In [ ]:
### `to_json`

将数据写入 JSON 格式文件。


In [ ]:
# 导出为 JSON 文件
dbn_data.to_json("GLBX-ESM2-20220606-ohlcv-1h.json")


In [ ]:
### `to_file`

将数据写入 DBN 文件。


In [ ]:
# 导出为 DBN 文件
dbn_data.to_file("GLBX-ESM2-20220606.ohlcv-1h.dbn.zst")


In [ ]:
### `to_ndarray`

* 将数据转换为 numpy N 维数组。
* 每个元素将包含二进制字段的 Python 表示作为一个 `Tuple`。


In [ ]:
# 导出为 numpy 数组
ndarray = dbn_data.to_ndarray()
ndarray


In [ ]:
### `to_parquet`

* 将数据写入 [Apache parquet](https://parquet.apache.org/) 格式文件。


In [ ]:
# 导出为 Apache Parquet 文件
dbn_data.to_parquet("GLBX-ESM2-20220606-ohlcv-1h.parquet")


In [ ]:
### `for` 循环

* 您可以使用标准的 Python `for` 循环来遍历 DBN 文件内容。


In [ ]:
# 让我们先加载一些数据
dbn_data = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    symbols=["ESM2"],
    schema="ohlcv-1h",
    start="2022-06-06",
    limit=3
)

# 包含 3 个小时K线
dbn_data.to_df()


In [ ]:
示例输出：*（非真实数据，只是输出格式示例）*

| ts_event | rtype | publisher_id | instrument_id | open | high | low | close | volume | symbol |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| 2022-06-06 00:00:00+00:00 | 34 | 1 | 3403 | 4109.50 | 4117.00 | 4105.50 | 4115.75 | 8541 | ESM2 |
| 2022-06-06 01:00:00+00:00 | 34 | 1 | 3403 | 4115.75 | 4122.75 | 4113.00 | 4122.25 | 14008 | ESM2 |
| 2022-06-06 02:00:00+00:00 | 34 | 1 | 3403 | 4122.25 | 4127.00 | 4120.75 | 4126.25 | 10150 | ESM2 |


In [ ]:
# 我们可以在 for 循环中使用 DBN 数据：
for bar in dbn_data:
    print(bar)   # 打印完整的K线数据
    break        # 故意中断以仅查看第一个K线


In [ ]:
示例输出：

```
OhlcvMsg {
    hd: RecordHeader {
        length: 14,
        rtype: Ohlcv1H,
        publisher_id: GlbxMdp3Glbx,
        instrument_id: 3403,
        ts_event: 1654473600000000000
    },
    open: 4109.500000000,
    high: 4117.000000000,
    low: 4105.500000000,
    close: 4115.750000000,
    volume: 4543
}
```


In [ ]:
for bar in dbn_data:
    print(f"K线开盘价: {bar.open}")  # 仅打印K线开盘价信息
    break                           # 故意中断以仅查看第一个K线


In [ ]:
示例输出：

`K线开盘价: 4108500000000`


In [ ]:
# 示例

## 下载 1 分钟 6E 数据


In [ ]:
from datetime import timedelta

import pandas as pd
import pytz


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [ ]:
# 设置
dataset="GLBX.MDP3"
symbol="6E.v.0"
stype_in="continuous"
schema="ohlcv-1m"
start="2025-01-01"
end="2025-01-05"


In [ ]:
# 检查美元成本
cost = client.metadata.get_cost(
    dataset=dataset,
    symbols=[symbol],
    stype_in=stype_in,
    schema=schema,
    start=start,
    end=end,
)

print(f"{cost:.2f}$")


In [ ]:
示例输出：

`0.01$`


In [ ]:
# 下载数据
data = client.timeseries.get_range(
    dataset=dataset,
    symbols=[symbol],
    stype_in=stype_in,
    schema=schema,
    start=start,
    end=end,
)

# 以 DBNStore 格式导出数据（CSV 数据比这大 10 倍）
data.to_file(f"{dataset}_{symbol}_{start}-{end}.{schema}.dbn.zst")


In [ ]:
# 清理并查看数据为 DataFrame
df = (
    data.to_df()
    .reset_index()
    .rename(columns={"ts_event": "datetime"})
    .drop(columns=["rtype", "publisher_id", "instrument_id"])

    # 列的良好顺序
    .reindex(columns=["symbol", "datetime", "open", "high", "low", "close", "volume"])

    # 将日期时间本地化为布拉迪斯拉发时区
    .assign(datetime = lambda df: pd.to_datetime(df["datetime"], utc=True))  # 标记为 UTC 日期时间
    .assign(datetime = lambda df: df["datetime"].dt.tz_convert(pytz.timezone("Europe/Bratislava")))  # 转换为布拉迪斯拉发时区

    # 添加 1 分钟，使日期时间表示K线的收盘时间（而不是开盘时间）
    .assign(datetime = lambda df: df["datetime"] + timedelta(minutes=1))
)

# 预览
print(len(df))
df.head(3)


In [ ]:
示例输出：*（非真实数据，只是输出格式示例）*

`2734`

| symbol | datetime | open | high | low | close | volume |
|:--|:--|:--|:--|:--|:--|:--|
| 6E.v.0 | 2025-01-02 00:01:00+01:00 | 1.03890 | 1.03930 | 1.03845 | 1.03905 | 291 |
| 6E.v.0 | 2025-01-02 00:02:00+01:00 | 1.03900 | 1.03900 | 1.03870 | 1.03880 | 311 |
| 6E.v.0 | 2025-01-02 00:03:00+01:00 | 1.03880 | 1.03890 | 1.03870 | 1.03885 | 140 |
